In [1]:
import ee
import geemap
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import folium

In [18]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AeaYSHDSp194NXdiMr2T3UrvBbDLYJIaHET1QfB4dn2QHzNs-OGldqMOlfM



Successfully saved authorization token.


In [19]:
#import shapefile
ke_boundary = ee.FeatureCollection("projects/connect-gaea/assets/gadm41_KEN_0")

# Get the geometry
geometry = ke_boundary.geometry()


In [20]:
def getNDVI(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['B8','B4']).rename("NDVI")
    image = image.addBands(ndvi)

    return(image)

In [21]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [28]:
Sentinel_data = ee.ImageCollection('COPERNICUS/S2').filterDate("2023-12-01","2023-12-31").filterBounds(ke_boundary) \
    .map(getNDVI).map(addDate).median()

In [29]:
color = ['red', 'yellow', 'green']
pallete = {"min":-1, "max":1, 'palette':color}

In [30]:
# initialize our map
map1 = geemap.Map()
map1.centerObject(ke_boundary, 8)
map1.addLayer(Sentinel_data.clip(ke_boundary).select('NDVI'), pallete, "NDVI")

map1.addLayerControl()
map1


Map(center=[0.5524335862141333, 37.84581141226501], controls=(WidgetControl(options=['position', 'transparent_…

In [31]:
task = ee.batch.Export.image.toDrive(
    image=Sentinel_data,
    description='NDVI_export',  # Specify a description
    folder='NDVI',  # Specify your Google Drive folder
    scale=30,  # Adjust scale as needed
    region=ke_boundary.geometry().bounds(),  # Define the region of interest
    fileFormat='GeoTIFF'
)
task.start()
